In [1]:
import numpy as np
import math

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

In [2]:
directory = '/n/home12/hongwei/HONGWEI/lagranto.era5/Simulation/run_2000_10yr/'

In [3]:
ds_01 = xr.open_dataset(directory+"traj_20000101.nc")
ds_04 = xr.open_dataset(directory+"traj_20000401.nc")
ds_07 = xr.open_dataset(directory+"traj_20000701.nc")
ds_10 = xr.open_dataset(directory+"traj_20001001.nc")


FileNotFoundError: [Errno 2] No such file or directory: b'/n/home12/hongwei/HONGWEI/lagranto.era5/Simulation/run_2000_10yr/traj_20000101.nc'

In [28]:
ds_07["BASEDATE"].shape

(14609, 1, 1, 6)

In [32]:
# set some parameters
N_column = 4 # [time, lon, lat, lev]

# for injection location
Nx = 36
Ny = 21
Nz = 5
N_parcel = len(ds_01['dimx_lon'])
print("N_parcel: ", N_parcel)
if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')


N_inject = 1 # how many injecting times

Nt = len(ds_01["time"]) # output every 6 hours for 10 years
print("Nt: ", Nt)

N_parcel:  3780
Nt:  14613


In [42]:
var = ds_01["lon"].values
var.shape

(14613, 1, 1, 3780)

In [43]:
var = np.moveaxis(var,0,3)
var.shape

(1, 1, 3780, 14613)

In [44]:
# loop for all traj files for month 01

Nt = ds_01["lon"].shape[0]

lifetime_01   = np.zeros((Nz,Ny,Nx, N_inject))
lifetime_N_01 = np.zeros((Nz,Ny,Nx, N_inject))
lifetime_S_01 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))
     
    var = ds_01["lon"].values
    var = np.moveaxis(var,0,3)
    data[:,:,1] = var[0,0,:,:] # lon
    
    var = ds_01["lat"].values
    var = np.moveaxis(var,0,3)
    data[:,:,2] = var[0,0,:,:] # lat
    
    var = ds_01["p"].values
    var = np.moveaxis(var,0,3)    
    data[:,:,3] = var[0,0,:,:] # lev
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                i_parcel = iy*(Nz*Nx) + iz*Nx + ix
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime   = np.zeros((Nz,Ny,Nx))
    lifetime_N = np.zeros((Nz,Ny,Nx))
    lifetime_S = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                Num_N = 0
                Num_S = 0
                for it in range(Nt):
                    Num = Num+1
                    
                    if data2[iz,iy,ix,it,2]>=0: 
                        Num_N = Num_N + 1
                    else:
                        Num_S = Num_S + 1
                    
                    if data2[iz,iy,ix,it,3]>=P_cont: 
                        lifetime[iz,iy,ix] = Num*3/24 # hour to day
                        lifetime_N[iz,iy,ix] = Num_N*3/24
                        lifetime_S[iz,iy,ix] = Num_S*3/24
                        break
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:]   = lifetime[:,index,:]
    lifetime_N[:,:,:] = lifetime_N[:,index,:]
    lifetime_S[:,:,:] = lifetime_S[:,index,:]
    data2[:,:,:,:,:]  = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_01[Nz,Ny,Nx,N_inject]
    lifetime_01[:,:,:,i_inject] = lifetime[:,:,:]
    lifetime_N_01[:,:,:,i_inject] = lifetime_N[:,:,:]
    lifetime_S_01[:,:,:,i_inject] = lifetime_S[:,:,:]
    


  0%|          | 0/1 [00:22<?, ?it/s]


NameError: name 'file1' is not defined

In [48]:
# save lifetime_01[Nz,Ny,Nx,N_inject] to Lifetime01.txt

with open('Lifetime_2000_01_01.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_01[iz,iy,ix,i_inject]))
                    f.write(',')
                    f.write(str(lifetime_N_01[iz,iy,ix,i_inject]))
                    f.write(',')
                    f.write(str(lifetime_S_01[iz,iy,ix,i_inject]))
                    f.write(',')
                    f.write('\n')    
f.close()

# with open('Lifetime_N_2000_01_01.txt', 'w') as f:
#     for i_inject in range(N_inject):
#         for ix in range(Nx):
#             for iy in range(Ny):
#                 for iz in range(Nz):
#                     f.write(str(lifetime_N_01[iz,iy,ix,i_inject]))
#                     f.write('\n')    
# f.close()

# with open('Lifetime_S_2000_01_01.txt', 'w') as f:
#     for i_inject in range(N_inject):
#         for ix in range(Nx):
#             for iy in range(Ny):
#                 for iz in range(Nz):
#                     f.write(str(lifetime_S_01[iz,iy,ix,i_inject]))
#                     f.write('\n')    
# f.close()